
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to_TF_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to_TF_Lite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

#Setup

### Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## Install required libraries
Be sure to click on the **restart** button after running this

In [ ]:
!pip install -U --pre tensorflow=="1.*"


Make sure you have `pycocotools` installed

# Imports & Configuration

In [ ]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import *


In [ ]:
%cd {DRIVE_DEV}
%cp part5-tflite-modelv9.zip /content/
%cd /
!unzip -o /content/part5-tflite-modelv9.zip

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(TF_LITE_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
  for _ in range(num_calibration_steps):
    # Get sample input data as a numpy array in a method of your choosing.
    yield [input]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()